In [171]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

In [213]:
class Net(pl.LightningModule):
    def __init__(self, batch_size=32, lr=0.01):
        super().__init__()
        self.fc1 = nn.Linear(187, 1)
        self.flat = nn.Flatten()

        self.batch_size = batch_size
        self.learning_rate = lr
    
    def forward(self, x):
        x = self.fc1(self.flat(x.float()))

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        return loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 0.001)

In [214]:
df_1= pd.read_csv("../data/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("../data/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y_train = torch.from_numpy(np.array(df_train[187].values).astype(np.int8))
X_train = torch.from_numpy(np.array(df_train[list(range(187))].values)[..., np.newaxis])

Y_test = torch.from_numpy(np.array(df_test[187].values).astype(np.int8))
X_test = torch.from_numpy(np.array(df_test[list(range(187))].values)[..., np.newaxis])

In [215]:
class NetDataset(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
    
    def setup(self, stage=None):
        self.train_dataset = TensorDataset(X_train, Y_train)
        self.test_dataset = TensorDataset(X_test, Y_test)

    def train_dataloader(self):
        train_loader = DataLoader(self.train_dataset)
        return train_loader
    
    def test_dataloader(self):
        test_loader = DataLoader(self.test_dataset)
        return test_loader

In [216]:
model = Net()

dataset = NetDataset()

trainer = pl.Trainer(auto_scale_batch_size=True, auto_lr_find=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [212]:
trainer.fit(model=model, datamodule=dataset)

/home/julian/miniconda3/envs/ml4h/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name | Type    | Params
---------------------------------
0 | fc1  | Linear  | 374   
1 | flat | Flatten | 0     
---------------------------------
374       Trainable params
0         Non-trainable params
374       Total params
0.001     Total estimated model params size (MB)
/home/julian/miniconda3/envs/ml4h/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/11641 [00:00<?, ?it/s] 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x187 and 1x187)